In [ ]:
import gym
import numpy as np
import tensorflow as tf

In [ ]:
env = gym.make("CartPole-v0")

In [ ]:
num_input_neurons = len(env.reset())
num_ouptut_neurons = env.action_space.n
hidden_layer_dimensions = [16, 32, 32]

learning_rate = 0.0001

In [ ]:
class DQN:
    def __init__(self,
                 session,
                 scope_name,
                 input_size,
                 hidden_layer_sizes,
                 output_size,
                 learning_rate):
        
        self.session = session
        self.scope_name = scope_name
        self.input_size = input_size
        self.hidden_layer_sizes = hidden_layer_sizes
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        with tf.variable_scope(self.scope_name):
            self.input = tf.placeholder(shape=[None, self.input_size],
                                        dtype=tf.float32)

            net = self.input
            for layer_num, layer_size in enumerate(self.hidden_layer_sizes):
                net = tf.layers.dense(net,
                                      layer_size,
                                      activation=tf.nn.relu)

            self.output = tf.layers.dense(net,
                                          self.output_size,
                                          activation=None)

            # Placeholder for expected q-values
            self.y = tf.placeholder(shape=[None, self.output_size], dtype=tf.float32)

            # Using the loss method provided by tf directly
            self.loss = tf.losses.mean_squared_error(self.y, self.output)

            self.optimizer = tf.train.AdamOptimizer(
                learning_rate=self.learning_rate).minimize(self.loss)
    
    def predict(self, state):
        return self.session.run(self.output,
                                feed_dict={self.input: np.reshape(state, [-1, self.input_size])})
    
    def update(self, state, y):
        return self.session.run([self.loss, self.optimizer],
                              feed_dict={
                                    self.input: state,
                                    self.y: y
                              })
    
    @staticmethod
    def create_copy_operations(source_scope, dest_scope):
        source_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=source_scope)
        dest_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=dest_scope)
        
        assert len(source_vars) == len(dest_vars)
        result = []
        
        for source_var, dest_var in zip(source_vars, dest_vars):
            result.append(dest_var.assign(source_var.value()))
        
        return result

In [ ]:
with tf.Session() as sess:

    main_dqn = DQN(session=sess,
                   scope_name="q_main",
                   input_size=num_input_neurons,
                   hidden_layer_sizes=hidden_layer_dimensions,
                   output_size=num_ouptut_neurons,
                   learning_rate=learning_rate)
    
    target_dqn = DQN(session=sess,
                     scope_name="q_target",
                     input_size=num_input_neurons,
                     hidden_layer_sizes=hidden_layer_dimensions,
                     output_size=num_ouptut_neurons,
                     learning_rate=learning_rate)
    
    sess.run(tf.global_variables_initializer())

    # Make them identical to begin with
    sess.run(DQN.create_copy_operations("q_main", "q_target"))
    
    # Experience replay and Training logic